In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), ".."))
main_dir = os.path.abspath('..')
os.chdir(main_dir)
sys.path.append(main_dir)

import numpy as np
import sympy as sp
import re, copy
from sklearn.model_selection import train_test_split
from PhysicsRegression import PhyReg

In [2]:
num_xy = 10000

x_to_fit = []
y_to_fit = []

data = np.load("./physical/data/To_QH_Data_0.npy")
E_a, E_r, LLT, L_Shell, MLT = data

assert not np.any(np.isnan(E_a))
assert not np.any(np.isnan(E_r))
assert not np.any(np.isnan(LLT))
assert not np.any(np.isnan(L_Shell))
assert not np.any(np.isnan(MLT))

LLT = LLT / 24 * 10
MLT = MLT / 24 * 10

#E_r / L_Shell, MLT, LLT
x_to_fit.append(np.array([
    L_Shell, MLT, LLT
]).copy().T)
y_to_fit.append(E_r.copy().reshape((-1, 1)))

for i in range(len(x_to_fit)):
    x_train, x_test, y_train, y_test = train_test_split(x_to_fit[i], y_to_fit[i], test_size=0.2, random_state=2024)
    x_to_fit[i] = x_train[:num_xy]
    y_to_fit[i] = y_train[:num_xy]

np.random.seed(2024)

In [ ]:
phyreg = PhyReg(
    path = "./model.pt",
    max_len=20000,
    device = "cuda:1"
)

phyreg.fit(
    x_to_fit, y_to_fit, 
    use_Divide=True, 
    use_MCTS=False, 
    use_GP=False, 
    use_pysr_init=True, 
    use_const_optimization=False,
    verbose=False,
    oracle_name="physical5",
    oracle_file="./physical/data/oracle_model_case5/",
    oracle_bs=512, oracle_lr=0.005, oracle_epoch=100,
    use_seperate_type=["id"],
    variable_scale=[-3, 10],
    save_oracle_model=True
)

In [7]:
best_gens = phyreg.best_gens
best_gens[0]["predicted_tree"] = "(-3.1894375591915707 * (0.784759547399746 * (((4.1298484646401 * cos((2.4997675312042835 * x_1))) - 4.912904529233774) * (((0.04341174536403441 * cos((-1.2005308667478727 * x_2))) + 0.7404857642898277) * 1/(((0.4022589976976407 - (0.38562638992630527 * cos(((1.2554995236686266 * x_1) - 9.465159092397949)))) * (((5.066631736805239 * (x_0)**2) - (-5.079372953230348 * x_0)) + -3.774508668184111)))))))"

best_gens = phyreg.constant_optimization(phyreg.best_gens, x_to_fit, y_to_fit)
phyreg.express_skeleton(best_gens, use_sp=True)

idx          : 0
expr skeleton: C_0*(C_1*cos(C_2*x_1) - C_3)*(C_4*cos(C_5*x_2) - C_6)/((C_7*cos(C_8*x_1 - C_9) - C_10)*(C_11*x_0**2 - C_12*x_0 + C_13))
constants    : -3361.9698 4.6951 1.2245 4.7611 1.4421 1.9287 56.5588 -18.9407 1.0335 8.9726 116.5368 1263.7229 885.3565 1067.1616

